# First Project for the Social Computing Course

We are gonna start to import all the stuff we need

In [ ]:
import tweepy
import os
import json
import pprint
import csv

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams